In [91]:
import pandas as pd

vehicles = pd.read_csv('results_original.csv')

vehicles = vehicles[vehicles.columns[1:]]

renaming = {
    'motorcycle': 'VP',
    'car': 'VP',
    'bus': 'CO',
    'truck': 'CO',
    'train': 'SR/RE'
}

vehicles['type'] = vehicles['type'].map(renaming)

print(vehicles['type'].unique())

vehicles

['VP' 'CO' 'SR/RE']


,type,initial_frame,detection_duration,lenght
0,VP,1,41,117.513428
1,CO,2,6,179.725708
2,CO,3,1,266.224365
3,VP,23,21,159.738953
4,VP,23,14,77.170410
...,...,...,...,...
5547,CO,107881,2,277.025146
5548,VP,107886,55,160.567810
5549,VP,107890,50,112.500366
5550,VP,107919,11,180.122818


# Calculo do tamanho do trecho (em metros)

Para calcular a extensão do trecho, vou usar o tamanho médio dos carros que passaram por ele (em pixels) e comparar com a extensão do trecho (em pixels).

O trecho escolhido possui 600 pixels de comprimento, e a média dos 3318 carros que passaram ao longo da hora selecionada é de 140 pixels. Logo o trecho é aproximadamente 4.5x o tamanho de um carro médio. Segundo as regras do estado de São Paulo, um carro médio possui 4.7m, levando isso em conta, o tamanho estimado do trecho é de 21m

In [94]:
road_lenght = 600 # in pixels

mean_car_lenght = vehicles[vehicles['type'] == 'VP'].lenght.mean() # in pixels

avg_car_size = 4.7 # in meters

estimated_road_lenght = road_lenght/mean_car_lenght * avg_car_size

print('Estimed road lenght: %.2f m '%estimated_road_lenght)

Estimed road lenght: 20.96 m 


# 2.b.  Apresente as contagens por classe de veículo e intervalo de 15 minutos em uma tabela e em um gráfico de colunas;

In [ ]:
def minutes_to_frames(minutes):
    fps = 30
    return min(minutes * 60 * fps, 107920) # 107920 is the number of frames in the video

intervals = [
    [0, minutes_to_frames(15), '09:00 - 09:15'],
    [minutes_to_frames(15), minutes_to_frames(30), '09:15 - 09:30'],
    [minutes_to_frames(30), minutes_to_frames(45), '09:30 - 09:45'],
    [minutes_to_frames(45), minutes_to_frames(60), '09:45 - 10:00'],
]



for interval in intervals:
    start, end, label = interval
    df = vehicles[(vehicles['initial_frame'] >= start) & (vehicles['initial_frame'] <= end)]
    df.loc[:, 'count'] = 1
    interval.append(df)

import plotly.express as px

def plot_vehicles_interval(interval):
    print('Time interval: ', interval[2])
    df = interval[3][['type', 'count']].groupby(['type']).count().reset_index()
    fig = px.bar(df, x='type', y='count', title=interval[2])
    fig.show()
    return df
    

In [55]:
i =1
interval = intervals[i]
plot_vehicles_interval(interval)

Time interval:  09:15 - 09:30


,type,count
0,CO,369
1,SR/RE,117
2,VP,902


# 2.c. Converta os volumes em veículos de passeio equivalentes pelos fatores de conversão do DNIT indicados a seguir e apresente os resultados em intervalos de 15 minutos em uma tabela e em um gráfico de colunas; 

In [63]:
convertion_factor = {
    'VP': 1,
    'CO': 1.5,
    'SR/RE': 2
}

def plot_vehicles_vp_eq_interval(interval):
    print('Time interval: ', interval[2])
    df = interval[3][['type', 'count']].groupby(['type']).count().reset_index()
    df['count'] = df['count'] * df['type'].map(convertion_factor)
    df['count (VP equivalent)'] = df['count']
    df = df[['type', 'count (VP equivalent)']]
    fig = px.bar(df, x='type', y='count (VP equivalent)', title=interval[2])
    fig.show()
    return df

i = 3
interval = intervals[i]
plot_vehicles_vp_eq_interval(interval)

Time interval:  09:45 - 10:00


,type,count (VP equivalent)
0,CO,582.0
1,SR/RE,274.0
2,VP,899.0


# 2.d.  Determine os fluxos horários nos intervalos de 15 minutos em veículos de passeio equivalentes e apresente os resultados em uma tabela e em um gráfico de colunas;

In [71]:
def flux(intervals):
    fluxes = []
    for interval in intervals:

        df = interval[3][['type', 'count']].groupby(['type']).count().reset_index()
        df['count'] = df['count'] * df['type'].map(convertion_factor)
        df['count (VP equivalent)'] = df['count']
        df = df[['type', 'count (VP equivalent)']]
        flux = df['count (VP equivalent)'].sum()
        fluxes.append([interval[2], flux])
    fluxes = pd.DataFrame(fluxes, columns=['interval', 'flux'])
    fig = px.bar(fluxes, x='interval', y='flux', title=interval[2])
    fig.show()
    return fluxes

fluxes = flux(intervals)
fluxes

,interval,flux
0,09:00 - 09:15,1667.5
1,09:15 - 09:30,1689.5
2,09:30 - 09:45,1697.0
3,09:45 - 10:00,1755.0


# 2.e. Calcule o fator hora-pico

In [75]:
valor_horario = fluxes['flux'].sum()

fator_hora_pico = valor_horario/fluxes['flux'].max() * (15/60)

print('Fator hora pico: %.2f'%fator_hora_pico)

Fator hora pico: 0.97


# 2.f. Calcule a densidade em intervalos de 15 minutos (em veículos de passeio equivalentes/km.faixa) e apresente os resultados em uma tabela e em um gráfico de colunas;

In [90]:
density = []
for interval in intervals:
    final_frame = interval[1]
    s1 = vehicles['initial_frame'] <= final_frame
    s2 = vehicles['initial_frame'] + vehicles['detection_duration'] >= final_frame
    s3 = s1 * s2
    cars = vehicles[s3]
    cars_converted = cars['type'].map(convertion_factor).sum()
    density.append([interval[2].split()[-1], cars_converted/estimated_road_lenght/3])

# fluxes = pd.DataFrame(density, columns=['interval', 'flux'])
# fig = px.bar(fluxes, x='interval', y='flux', title=interval[2])
# fig.show()

estimated_road_lenght

nan

In [84]:
cars

,type,initial_frame,detection_duration,lenght
5548,VP,107886,55,160.567810
5549,VP,107890,50,112.500366
5550,VP,107919,11,180.122818
